# Data Cleaning

We have six datasets: 
1. Participant data
2. Participant task interaction data
3. Personality test of participants
4. Tasks
5. Post Study questionnaire data
6. Messages data

In [1]:
# Imports
import pandas as pd

## Participants cleaning

In [9]:
participant = pd.read_csv('./data/participant_rows.csv')
participant = participant.drop(columns=['password'], axis=1)
participant.head()


,id,created_at,name,email,age,gender,education,occupation,nationality,frequency_usage,english_fluency,ai_usage,consent,familiarity
0,1,2025-08-11 16:39:58.335908+00,Shajia Mithani,shajia.mithani@gmail.com,28,female,university,social,pakistan,multiple-times,fluent-c1,"[{""label"":""Teaching"",""value"":""teaching""}]",True,"{""1"":true,""2"":true,""3"":true,""4"":false,""5"":true..."
1,2,2025-08-11 16:54:37.71803+00,Tauqeer Ahmed Iqbal Ahmed Shaikh,tauqeer.shaikh@stud.fra-uas.de,24,male,university,computing,india,constantly,fluent-c1,"[{""label"":""Work (e.g. writing, coding)"",""value...",True,"{""1"":true,""2"":true,""3"":true,""4"":true,""5"":true,..."
2,3,2025-08-11 17:10:42.112849+00,Hamza Asaad,hamza.asaad@stud.fra-uas.de,26,male,university,computing,pakistan,always,fluent-c1,"[{""label"":""Work (e.g. writing, coding)"",""value...",True,"{""1"":true,""2"":false,""3"":false,""4"":false,""5"":fa..."
3,4,2025-08-11 17:26:05.462118+00,rafia Basheer,rafia.ahamed-basheer@stud.fra-uas.de,33,female,university,engineering,india,often,intermediate-b2,"[{""label"":""Learning / Studying"",""value"":""learn...",True,"{""1"":true,""2"":true,""3"":true,""4"":true,""5"":true,..."
4,5,2025-08-11 18:52:21.661286+00,Adesh Shirke,adesh.shirke@stud.fra-uas.de,27,male,post-secondary,engineering,india,always,fluent-c1,"[{""label"":""Research / Information Gathering"",""...",True,"{""1"":true,""2"":false,""3"":false,""4"":false,""5"":tr..."


In [10]:
participant_task_interaction = pd.read_csv('./data/participant_task_interaction_rows.csv')
participant_task_interaction.head()


,id,started_at,ended_at,ai_tool,participant_id,task_id
0,1,2025-08-11 16:43:46.321746+00,2025-08-11 16:43:46.588334+00,GPT-4o,1,1
1,2,2025-08-11 16:47:04.536261+00,NaN,GPT-4o,1,2
2,3,2025-08-11 16:48:15.553518+00,2025-08-11 16:49:21.11072+00,GPT-4o,1,3
3,4,2025-08-11 16:48:39.841189+00,NaN,GPT-4o,1,3
4,5,2025-08-11 16:49:07.215491+00,NaN,GPT-4o,1,3


In [17]:
message = pd.read_csv('./data/message_rows.csv')
message.head()

,id,created_at,interaction_id,sender,content
0,1,2025-08-11 16:43:00.914+00,1,user,"standing on the balcony, overlooking the farms..."
1,2,2025-08-11 16:43:39.07+00,1,ai,/static/images/image_20250811_164346_345588_dd...
2,3,2025-08-11 16:46:06.323+00,2,user,"cream coloured walls, with hanging plants in s..."
3,4,2025-08-11 16:46:48.415+00,2,ai,/static/images/image_20250811_164704_561704_f5...
4,5,2025-08-11 16:48:14.409+00,3,user,i do the cleaning my husband does the cooking


In [21]:
merged_df = participant_task_interaction.merge(
    message,
    left_on="id",          # id in PTI
    right_on="interaction_id",  # matches in message
    how="inner"
)

# Sort for chronological conversations
merged_df = merged_df.sort_values(by=["participant_id", "task_id", "created_at"])

# Optional: If you want conversations grouped as text per participant/task
grouped_conversations = (
    merged_df.groupby(["participant_id", "task_id"])
    .apply(lambda x: "\n".join(f"{row['sender']}: {row['content']}" for _, row in x.iterrows()))
    .reset_index(name="conversation")
)

grouped_conversations.to_csv('./data/grouped_conversations.csv', index=False)

C:\Users\Aliza\AppData\Local\Temp\ipykernel_29372\3472320614.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: "\n".join(f"{row['sender']}: {row['content']}" for _, row in x.iterrows()))


In [26]:
# Group by participant-task and collect messages into a structured list
structured_conversations = (
    merged_df.groupby(["participant_id", "task_id"])
    .apply(lambda x: [
        {
            "timestamp": row["created_at"],
            "sender": row["sender"],
            "content": row["content"]
        }
        for _, row in x.iterrows()
    ])
    .reset_index(name="conversation")
)
structured_conversations['conversation'][0]

C:\Users\Aliza\AppData\Local\Temp\ipykernel_29372\3276724639.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: [


[{'timestamp': '2025-08-11 16:43:00.914+00',
  'sender': 'user',
  'content': 'standing on the balcony, overlooking the farms at cameron highlands.. clouds hanging very low, passing through the trees as it rains'},
 {'timestamp': '2025-08-11 16:43:39.07+00',
  'sender': 'ai',
  'content': '/static/images/image_20250811_164346_345588_dd8e83db-effe-43cf-80c8-13cf4fa8bf5b.png'}]